# Federal Retention Notebook

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, GridSearchCV, cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import classification_report, r2_score, make_scorer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import RidgeCV, BayesianRidge, LassoCV, RANSACRegressor, OrthogonalMatchingPursuit, ElasticNetCV, Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from collections import defaultdict
from sklearn.neighbors import KNeighborsRegressor

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(style="darkgrid")

_____

# I. Importing, Exploring, and Transforming the Data

In [ ]:
df = pd.read_csv("./Agency_Full_Clean_2.csv")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
df["engagement"] = (df["40"] + df["69"] + df["71"])/3

In [ ]:
df1 = df[['attrition_rate', 'average_salary', 'percent_advance_degrees', 'percent_female', 'engagement']]
X_small = df[['average_salary', 'percent_advance_degrees', 'percent_female', 'engagement']]
X_big = df.drop(columns=["agency", "agency_code", "employment_count", "quit_count", "year", "attrition_rate", "average_service", "engagement"])
y = df.attrition_rate
y = np.asarray(y)

In [ ]:
X_small.describe()

In [ ]:
axes = sns.pairplot(df1)

In [ ]:
poly_features= PolynomialFeatures(degree=2)
X_s_poly = poly_features.fit_transform(X_small)

In [ ]:
scaler = StandardScaler()
X_small_scaled = scaler.fit_transform(X_small)
X_small_poly_scaled = scaler.fit_transform(X_s_poly)

In [ ]:
from yellowbrick.target import FeatureCorrelation

fig = plt.figure(figsize=(10,8))
features = ['average_salary', 'percent_advance_degrees', 'percent_female', 'engagement']
vf = FeatureCorrelation(labels=features)
vf.fit(X_small_scaled, y)      
vf.show()     
fig.savefig('small_features.png')

In [ ]:
features = ['average_salary', 'percent_advance_degrees', 'percent_female', 'engagement']
vf = FeatureCorrelation(method='mutual_info-regression', labels=features)
vf.fit(X_small_scaled, y)      
vf.show()   

In [ ]:
print(X_small_scaled.shape)
print(X_small_poly_scaled.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_small_scaled, y, test_size=0.2, random_state=42)

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_small_poly_scaled, y, test_size=0.2, random_state=42)

_____

# II. Creating, Testing, and Visualizing the Small Model

In [ ]:
# Initial Testing Small Scaled Model (no polynomials)

ridge = RidgeCV()
ridge.fit(X_train, y_train)
print("Ridge Train Data R2 score : %.3f" %ridge.score(X_train,y_train))
print("Ridge Test Data R2 score : %.3f" % ridge.score(X_test, y_test))

en = ElasticNetCV()
en.fit(X_train, y_train)
print("ElasticNet Train Data R2 score : %.3f" %en.score(X_train,y_train))
print("ElasticNet Test Data R2 score : %.3f" %en.score(X_test, y_test))

bayes = BayesianRidge()
bayes.fit(X_train, y_train)
print("Bayes Train Data R2 score : %.3f" %bayes.score(X_train,y_train))
print("Bayes Test Data R2 score : %.3f" %bayes.score(X_test, y_test))

lasso = LassoCV()
lasso.fit(X_train, y_train)
print("Lasso Train Data R2 score : %.3f" %lasso.score(X_train,y_train))
print("Lasso Test Data R2 score : %.3f" %lasso.score(X_test, y_test))

RANSAC = RANSACRegressor()
RANSAC.fit(X_train, y_train)
print("RANSAC Train Data R2 score : %.3f" %RANSAC.score(X_train,y_train))
print("RANSAC Test Data R2 score : %.3f" %RANSAC.score(X_test, y_test))

OMP = OrthogonalMatchingPursuit()
OMP.fit(X_train, y_train)
print("OMP Train Data R2 score : %.3f" %OMP.score(X_train,y_train))
print("OMP Test Data R2 score : %.3f" %OMP.score(X_test, y_test))

SVReg = SVR()
SVReg.fit(X_train, y_train)
print("SVR Train Data R2 score : %.3f" %SVReg.score(X_train,y_train))
print("SVR Test Data R2 score : %.3f" %SVReg.score(X_test, y_test))

GBReg = GradientBoostingRegressor()
GBReg.fit(X_train, y_train)
print("GBReg Train Data R2 score : %.3f" %GBReg.score(X_train,y_train))
print("GBReg Test Data R2 score : %.3f" %GBReg.score(X_test, y_test))

RFReg = RandomForestRegressor()
RFReg.fit(X_train, y_train)
print("RFReg Train Data R2 score : %.3f" %RFReg.score(X_train,y_train))
print("RFReg Test Data R2 score : %.3f" %RFReg.score(X_test, y_test))

In [ ]:
r2_val = []
for K in range(12):
    K = K+1
    ne = KNeighborsRegressor(n_neighbors = K, weights='distance', metric='braycurtis')
    ne.fit(X_train, y_train)
    r2 = ne.score(X_test, y_test)  
    r2_val.append(r2)
    print('R2 k= ' , K ,'for test data:' , r2)

In [ ]:
nn = Sequential()
nn.add(Dense(32, activation='relu', input_dim=4))
nn.add(Dense(16, activation='relu'))
nn.add(Dense(8, activation='relu'))
nn.add(Dense(1, activation='linear'))

RMSprop = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
nn.compile(loss='mean_squared_error', optimizer=RMSprop, metrics=['mean_squared_error'])
nn.fit(X_train, y_train, epochs=70, batch_size=1, verbose=0, shuffle=True)
preds133 = nn.predict(X_train)
print("Train Data R2 score : %.3f" % r2_score(y_train,preds133))
preds123 = nn.predict(X_test)
print("Test Data R2 score : %.3f" % r2_score(y_test,preds123))

In [ ]:
# Initial Testing Small Model with Polynomials

ridge = RidgeCV()
ridge.fit(X_train1, y_train1)
print("Ridge Train Data R2 score : %.3f" %ridge.score(X_train1,y_train1))
print("Ridge Test Data R2 score : %.3f" % ridge.score(X_test1, y_test1))

en = ElasticNetCV()
en.fit(X_train1, y_train1)
print("ElasticNet Train Data R2 score : %.3f" %en.score(X_train1,y_train1))
print("ElasticNet Test Data R2 score : %.3f" %en.score(X_test1, y_test1))

bayes = BayesianRidge()
bayes.fit(X_train1, y_train1)
print("Bayes Train Data R2 score : %.3f" %bayes.score(X_train1,y_train1))
print("Bayes Test Data R2 score : %.3f" %bayes.score(X_test1, y_test1))

lasso = LassoCV()
lasso.fit(X_train1, y_train1)
print("Lasso Train Data R2 score : %.3f" %lasso.score(X_train1,y_train1))
print("Lasso Test Data R2 score : %.3f" %lasso.score(X_test1, y_test1))

RANSAC = RANSACRegressor()
RANSAC.fit(X_train1, y_train1)
print("RANSAC Train Data R2 score : %.3f" %RANSAC.score(X_train1,y_train1))
print("RANSAC Test Data R2 score : %.3f" %RANSAC.score(X_test1, y_test1))

OMP = OrthogonalMatchingPursuit()
OMP.fit(X_train1, y_train1)
print("OMP Train Data R2 score : %.3f" %OMP.score(X_train1,y_train1))
print("OMP Test Data R2 score : %.3f" %OMP.score(X_test1, y_test1))

SVR1 = SVR()
SVR1.fit(X_train1, y_train1)
print("SVR Train Data R2 score : %.3f" %SVR1.score(X_train1,y_train1))
print("SVR Test Data R2 score : %.3f" %SVR1.score(X_test1, y_test1))

GBReg = GradientBoostingRegressor()
GBReg.fit(X_train1, y_train1)
print("GBReg Train Data R2 score : %.3f" %GBReg.score(X_train1,y_train1))
print("GBReg Test Data R2 score : %.3f" %GBReg.score(X_test1, y_test1))

RFReg = RandomForestRegressor()
RFReg.fit(X_train1, y_train1)
print("RFReg Train Data R2 score : %.3f" %RFReg.score(X_train1,y_train1))
print("RFReg Test Data R2 score : %.3f" %RFReg.score(X_test1, y_test1))

In [ ]:
r2_val = []
for K in range(12):
    K = K+1
    ne = KNeighborsRegressor(n_neighbors = K, weights='distance', metric='braycurtis')
    ne.fit(X_train1, y_train1)
    r2 = ne.score(X_test1, y_test1)  
    r2_val.append(r2)
    print('R2 k= ' , K ,'for test data:' , r2)

In [ ]:
nn1 = Sequential()
nn1.add(Dense(32, activation='relu', input_dim=15))
nn1.add(Dense(16, activation='relu'))
nn1.add(Dense(8, activation='relu'))
nn1.add(Dense(1, activation='linear'))

RMSprop = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
nn1.compile(loss='mean_squared_error', optimizer=RMSprop, metrics=['mean_squared_error'])
nn1.fit(X_train1, y_train1, epochs=70, batch_size=1, verbose=0, shuffle=True)
preds134 = nn1.predict(X_train1)
print("Train Data R2 score : %.3f" % r2_score(y_train1,preds134))
preds125 = nn1.predict(X_test1)
print("Test Data R2 score : %.3f" % r2_score(y_test1,preds125))

In [ ]:
# Implement Grid Search for Selected Models on Scaled (but not Polynomial) Data

def grid_search(X, y, model, param_grid, name='', cv=KFold(n_splits=5,shuffle=True),
                scoring='r2', verbose=False, n_jobs=-1):
    '''Performs a grid search over param_grid and returns the best model'''
    GSmodel = GridSearchCV(model, cv=cv, param_grid=param_grid, scoring=scoring, n_jobs=n_jobs)
    GSmodel = GSmodel.fit(X, y)
    if (verbose):
        print(name+"best params:")
        print(GSmodel.best_params_)
        print(str(scoring)+":")
        print(-1*GSmodel.best_score_)
    return GSmodel.best_estimator_

def make_CV_models(X, y):
    '''performs grid searches to find all the best models for dataset X, y'''
    model_dict = {
            'KRR'    : grid_search(X, y, KernelRidge(), param_grid={"alpha": np.logspace(-10, 2, 100), "gamma": np.logspace(-10, -1, 50), "kernel" : ['rbf']}),
            'SVR'   : grid_search(X, y, SVR(), param_grid={"C": np.logspace(-1, 4, 20), "epsilon": np.logspace(-2, 2, 20)}),
            'Ridge' : grid_search(X, y, Ridge(), param_grid={"alpha": np.logspace(-6, 6, 100)} ),
            'Lasso' : grid_search(X, y, Lasso(max_iter = 20000), param_grid={"alpha": np.logspace(-2, 6, 50)} ),
            'BR'    : grid_search(X, y, BayesianRidge(), param_grid={"alpha_1": np.logspace(-13,-5,10),"alpha_2": np.logspace(-9,-3,10), "lambda_1": np.logspace(-10,-5,10),"lambda_2": np.logspace(-11,-4,10)}) ,
            'GBoost': grid_search(X, y, GradientBoostingRegressor(), param_grid={"n_estimators": np.linspace(5, 350, 50).astype('int')} ),
            'RF'    : grid_search(X, y, RandomForestRegressor(), param_grid={"n_estimators": np.linspace(5, 100, 30).astype('int')}, ),
            'kNN'   : grid_search(X, y, KNeighborsRegressor(), param_grid={"n_neighbors": np.linspace(2,20,18).astype('int')} )}

    return model_dict

In [ ]:
make_CV_models(X_train, y_train)

In [ ]:
model1 = KernelRidge(alpha=0.040370172585965494, coef0=1, degree=3, gamma=0.1,
             kernel='rbf', kernel_params=None)
model1.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %model1.score(X_train,y_train))
print("Test Data R2 score : %.3f" %model1.score(X_test, y_test))

In [ ]:
model2 = SVR(C=12.742749857031335, cache_size=200, coef0=0.0, degree=3,
     epsilon=0.7847599703514611, gamma='scale', kernel='rbf', max_iter=-1,
     shrinking=True, tol=0.001, verbose=False)
model2.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %model2.score(X_train,y_train))
print("Test Data R2 score : %.3f" %model2.score(X_test, y_test))

In [ ]:
model3 = Ridge(alpha=57.223676593502205, copy_X=True, fit_intercept=True, max_iter=None,
       normalize=False, random_state=None, solver='auto', tol=0.001)
model3.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %model3.score(X_train,y_train))
print("Test Data R2 score : %.3f" %model3.score(X_test, y_test))

In [ ]:
model4 = Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=20000,
       normalize=False, positive=False, precompute=False, random_state=None,
       selection='cyclic', tol=0.0001, warm_start=False)
model4.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %model4.score(X_train,y_train))
print("Test Data R2 score : %.3f" %model4.score(X_test, y_test))

In [ ]:
model5 = BayesianRidge(alpha_1=5.994842503189421e-12, alpha_2=0.001, alpha_init=None,
               compute_score=False, copy_X=True, fit_intercept=True,
               lambda_1=1e-05, lambda_2=0.0001, lambda_init=None, n_iter=300,
               normalize=False, tol=0.001, verbose=False)
model5.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %model5.score(X_train,y_train))
print("Test Data R2 score : %.3f" %model5.score(X_test, y_test))

In [ ]:
model6 = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                           init=None, learning_rate=0.1, loss='ls', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=75,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0, warm_start=False)
model6.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %model6.score(X_train,y_train))
print("Test Data R2 score : %.3f" %model6.score(X_test, y_test))

In [ ]:
model7 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=96, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)
model7.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %model7.score(X_train,y_train))
print("Test Data R2 score : %.3f" %model7.score(X_test, y_test))

In [ ]:
model8 = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=12, p=2,
                     weights='uniform')
model8.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %model8.score(X_train,y_train))
print("Test Data R2 score : %.3f" %model8.score(X_test, y_test))

In [ ]:
# Small Model Conclusion -- Best Model = KNeighbors Regression

small_model = KNeighborsRegressor(n_neighbors = 5, weights='distance', metric='braycurtis')
small_model.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %small_model.score(X_train,y_train))
print("Test Data R2 score : %.3f" %small_model.score(X_test, y_test)) 

In [ ]:
from yellowbrick.regressor import PredictionError

fig = plt.figure()
v1 = PredictionError(small_model)
v1.fit(X_train, y_train)
v1.score(X_test, y_test)
v1.show()
fig.savefig('small_PE.png')

In [ ]:
from sklearn.model_selection import KFold
from yellowbrick.model_selection import CVScores

fig = plt.figure()
cv = KFold(n_splits=10, random_state=42)
v2 = CVScores(small_model, cv=cv, scoring='r2')
v2.fit(X_train, y_train)
v2.show()  
fig.savefig('small_CV.png')

In [ ]:
from yellowbrick.regressor import ResidualsPlot

fig = plt.figure()
v3 = ResidualsPlot(small_model)
v3.fit(X_train, y_train) 
v3.score(X_test, y_test) 
v3.show() 
fig.savefig('small_Res.png')

In [ ]:
from yellowbrick.model_selection import LearningCurve

fig = plt.figure()
v4 = LearningCurve(small_model, scoring='r2')
v4.fit(X_train, y_train)      
v4.show()  
fig.savefig('small_LC.png')

In [ ]:
from yellowbrick.model_selection import ValidationCurve

param_range = np.arange(1, 10, 1)
v5 = ValidationCurve(small_model, param_name="n_neighbors",
    param_range=param_range, cv=6, scoring="r2", n_jobs=-1)
v5.fit(X_train, y_train)
v5.show()

In [ ]:
from yellowbrick.regressor import CooksDistance

v6 = CooksDistance()
v6.fit(X_train, y_train)
v6.show()

___

# III. Creating, Testing, and Visualizing the Big Model

In [ ]:
X_big_scaled = scaler.fit_transform(X_big)
X_train, X_test, y_train, y_test = train_test_split(X_big_scaled, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_big.columns

In [ ]:
features1 = ['average_salary', 'percent_advance_degrees', 'percent_female', '1', '2',
       '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27',
       '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
       '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51',
       '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63',
       '64', '65', '66', '67', '68', '69', '70', '71']

fig = plt.figure(figsize=(10,16))
vf = FeatureCorrelation(labels=features1)
vf.fit(X_big_scaled, y)      
vf.show()
fig.savefig('big_features.png')

In [ ]:
# Initial Testing Large Scaled Model (no polynomials)

ridge1 = RidgeCV()
ridge1.fit(X_train, y_train)
print("Ridge Train Data R2 score : %.3f" %ridge1.score(X_train,y_train))
print("Ridge Test Data R2 score : %.3f" % ridge1.score(X_test, y_test))

en1 = ElasticNetCV()
en1.fit(X_train, y_train)
print("ElasticNet Train Data R2 score : %.3f" %en1.score(X_train,y_train))
print("ElasticNet Test Data R2 score : %.3f" %en1.score(X_test, y_test))

bayes1 = BayesianRidge()
bayes1.fit(X_train, y_train)
print("Bayes Train Data R2 score : %.3f" %bayes1.score(X_train,y_train))
print("Bayes Test Data R2 score : %.3f" %bayes1.score(X_test, y_test))

lasso1 = LassoCV()
lasso1.fit(X_train, y_train)
print("Lasso Train Data R2 score : %.3f" %lasso1.score(X_train,y_train))
print("Lasso Test Data R2 score : %.3f" %lasso1.score(X_test, y_test))

RANSAC1 = RANSACRegressor()
RANSAC1.fit(X_train, y_train)
print("RANSAC Train Data R2 score : %.3f" %RANSAC1.score(X_train,y_train))
print("RANSAC Test Data R2 score : %.3f" %RANSAC1.score(X_test, y_test))

OMP1 = OrthogonalMatchingPursuit()
OMP1.fit(X_train, y_train)
print("OMP Train Data R2 score : %.3f" %OMP1.score(X_train,y_train))
print("OMP Test Data R2 score : %.3f" %OMP1.score(X_test, y_test))

SVReg1 = SVR()
SVReg1.fit(X_train, y_train)
print("SVR Train Data R2 score : %.3f" %SVReg1.score(X_train,y_train))
print("SVR Test Data R2 score : %.3f" %SVReg1.score(X_test, y_test))

GBReg1 = GradientBoostingRegressor()
GBReg1.fit(X_train, y_train)
print("GBReg Train Data R2 score : %.3f" %GBReg1.score(X_train,y_train))
print("GBReg Test Data R2 score : %.3f" %GBReg1.score(X_test, y_test))

RFReg1 = RandomForestRegressor()
RFReg1.fit(X_train, y_train)
print("RFReg Train Data R2 score : %.3f" %RFReg1.score(X_train,y_train))
print("RFReg Test Data R2 score : %.3f" %RFReg1.score(X_test, y_test))

In [ ]:
r2_val1 = []
for K in range(12):
    K = K+1
    ne1 = KNeighborsRegressor(n_neighbors = K, weights='distance', metric='braycurtis')
    ne1.fit(X_train, y_train)
    r2 = ne1.score(X_test, y_test)  
    r2_val1.append(r2)
    print('R2 k= ' , K ,'for test data:' , r2)

In [ ]:
make_CV_models(X_train, y_train)

In [ ]:
big_model = SVR(C=6.951927961775605, cache_size=200, coef0=0.0, degree=3,
     epsilon=0.016237767391887217, gamma='scale', kernel='rbf', max_iter=-1,
     shrinking=True, tol=0.001, verbose=False)
big_model.fit(X_train, y_train)
print("Train Data R2 score : %.3f" %big_model.score(X_train,y_train))
print("Test Data R2 score : %.3f" %big_model.score(X_test, y_test))

In [ ]:
nn_big = Sequential()
nn_big.add(Dense(32, activation='relu', input_dim=74))
nn_big.add(Dropout(0.2))
nn_big.add(Dense(16, activation='relu'))
nn_big.add(Dropout(0.2))
nn_big.add(Dense(8, activation='relu'))
nn_big.add(Dense(1))

Adam = tf.keras.optimizers.Adam(learning_rate=0.001)
nn_big.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mean_squared_error'])
model_output = nn_big.fit(X_train, y_train, epochs=30, batch_size=1, verbose=0, shuffle=False)

In [ ]:
pt = nn_big.predict(X_train)
pt1 = nn_big.predict(X_test)
print("R-Squared for the Training Data: %.3f" % r2_score(y_train,pt))
print("R-Squared for the Test Data: %.3f" % r2_score(y_test,pt1))

In [ ]:
fig = plt.figure()
v1 = PredictionError(big_model)
v1.fit(X_train, y_train)
v1.score(X_test, y_test)
v1.show()
fig.savefig('big_PE.png')

In [ ]:
cv = KFold(n_splits=10, random_state=42)
v2 = CVScores(big_model, cv=cv, scoring='r2')
v2.fit(X_train, y_train)
v2.show()  

In [ ]:
fig = plt.figure()
v3 = ResidualsPlot(big_model)
v3.fit(X_train, y_train) 
v3.score(X_test, y_test) 
v3.show() 
fig.savefig('big_Res.png')

In [ ]:
fig = plt.figure()
v4 = LearningCurve(big_model, scoring='r2')
v4.fit(X_train, y_train)      
v4.show()  
fig.savefig('big_LC.png')

In [ ]:
v6 = CooksDistance()
v6.fit(X_train, y_train)
v6.show()

## Appendix: H2O AutoML

In [ ]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

In [ ]:
frame = h2o.H2OFrame(df1)
trainA, testA = frame.split_frame(ratios=[.8])
x = frame.columns
y = "attrition_rate"
x.remove(y)

In [ ]:
aml = H2OAutoML(max_runtime_secs=1500, seed=1, verbosity="info")
aml.train(x=x, y=y, training_frame=trainA)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
aml.leader.r2()